In [ ]:
import pandas as pd

import ipywidgets as widgets
from ipywidgets import interact

from src.data_load import load_tables, load_artificial_instance, load_distances
from src.filtering import filter_labors_by_city, filter_labors_by_date
from src.plotting import plot_metrics_comparison, plot_gantt_labors_by_driver, plot_results
from src.metrics import collect_vt_metrics_range, show_day_report_dayonly, compute_indicators, collect_results_to_df, compute_metrics_with_moves
from src.config import *

data_path = '../data'
instance = 'inst3'
directorio_df, labors_raw_df, cities_df, duraciones_df, valid_cities = load_tables(data_path, generate_labors=False)
labors_real_df = load_artificial_instance(data_path, instance, labors_raw_df)
dist_dict = load_distances(data_path, 'real', instance)

fechas = pd.date_range("2025-09-08", "2025-09-13").strftime("%Y-%m-%d").tolist()

# Cargar resultados

In [ ]:
labors_hist_df, moves_hist_df = collect_results_to_df(data_path, instance, fechas, assignment_type='historic')
labors_hist_df = labors_hist_df[pd.notna(labors_hist_df['address_point'])]
labors_algo_df, moves_algo_df = collect_results_to_df(data_path, instance, fechas, assignment_type='algorithm')
labors_algo_df = labors_algo_df[pd.notna(labors_algo_df['address_point'])]
moves_algo_df = moves_algo_df[moves_algo_df['service_id'].isin(moves_hist_df['service_id'].unique().tolist())]

In [12]:
metrics_algo_df =  compute_metrics_with_moves(
    labors_algo_df,
    moves_algo_df,
    fechas = fechas,
    # start_date=fechas[0],
    # end_date=fechas[-1],
    dist_dict=dist_dict,
    workday_hours=8,
    city='149',
    skip_weekends=False,
    assignment_type='algorithm'
)

metrics_algo_df

,day,vt_count,num_drivers,labores_por_conductor,utilizacion_promedio_%,tiempo_extra,total_distance,driver_move_distance
0,2025-09-08,75,20,3.750,30.8,882.4,1347.9239,414.357199
1,2025-09-09,69,16,4.312,39.1,871.9,828.5413,393.436820
2,2025-09-10,72,19,3.789,39.2,875.8,1184.7515,498.471018
3,2025-09-11,70,20,3.500,29.7,1020.2,623.5749,412.605152
4,2025-09-12,67,18,3.722,32.1,1069.1,668.8300,443.113959
5,2025-09-13,76,20,3.800,32.9,1031.6,699.9027,479.548224


# Results comparison

In [ ]:
metricas = ['vt_count', 'num_drivers', "labores_por_conductor", 'total_distance', 'tiempo_extra', 'driver_move_distance']
# metricas = ['num_drivers', 'total_distance', 'tiempo_extra', 'driver_move_distance']

def plot_metrics_comparison_interactive(
    labors_hist_df: pd.DataFrame,
    moves_hist_df: pd.DataFrame,
    labors_algo_df: pd.DataFrame,
    moves_algo_df: pd.DataFrame,
    cities: list[str],
    metricas,
    dist_dict: dict,
    fechas: tuple[str, str]
):
    """
    Interactive version of plot_metrics_comparison with ipywidgets.
    Allows selecting the city dynamically from a dropdown.
    """

    def _plot_for_city(city: str):
        # Call your existing function
        figs = plot_metrics_comparison(
            labors_hist_df=labors_hist_df,
            moves_hist_df=moves_hist_df,
            labors_algo_df=labors_algo_df,
            moves_algo_df=moves_algo_df,
            city=city,
            metricas=metricas,
            dist_dict=dist_dict,
            fechas=fechas
        )
        # return figs

    # Create dropdown
    dropdown = widgets.Dropdown(
        options=cities,
        value=cities[0],  # default city
        description="Ciudad:",
        style={"description_width": "initial"},
        layout=widgets.Layout(width="300px")
    )

    # Link dropdown with plotting function
    interact(_plot_for_city, city=dropdown)

plot_metrics_comparison_interactive(
    labors_hist_df,
    moves_hist_df,
    labors_algo_df,
    moves_algo_df,
    cities=valid_cities,  # put the city codes here
    metricas=metricas,
    dist_dict=dist_dict,
    fechas=("2025-09-08", "2025-09-13")
)

interactive(children=(Dropdown(description='Ciudad:', layout=Layout(width='300px'), options=('149', '1', '126'…

### Pruebas adicionales

In [10]:
labors_algo_df[(labors_algo_df['city']=='149') &
              (labors_algo_df['schedule_date'].dt.day==9) &
              (labors_algo_df['labor_category']=='VEHICLE_TRANSPORTATION')]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
172,201820,293454.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 06:00:00-05:00,NaN,Yeisson Beltran Garzon,2025-09-09 05:30:00-05:00,2025-09-09 06:03:24.033399-05:00,...,123975.0,POINT (-74.10727299999999 4.6525963),123976.0,POINT (-74.1145559 4.6716483),149,COMPLETED,POINT (-74.10727299999999 4.6525963),POINT (-74.1145559 4.6716483),2025-09-09,16
173,204964,296831.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 07:00:00-05:00,NaN,Julio Cesar Parra,2025-09-09 06:30:00-05:00,2025-09-09 07:03:43.853904-05:00,...,123805.0,POINT (-74.0574336 4.708063999999999),10.0,POINT (-74.075015 4.69416),149,COMPLETED,POINT (-74.0574336 4.708063999999999),POINT (-74.075015 4.69416),2025-09-09,16
174,205425,297314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 13:30:00-05:00,NaN,José Ricardo Pérez Alarcón,2025-09-09 13:00:00-05:00,2025-09-09 13:39:20.784593-05:00,...,125179.0,POINT (-74.03973859999999 4.7061332),28798.0,POINT (-74.08720430000001 4.6760977),149,FAILED,POINT (-74.03973859999999 4.7061332),POINT (-74.08720430000001 4.6760977),2025-09-09,16
175,205461,297351.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 08:30:00-05:00,NaN,Manuel Romero,2025-09-09 08:00:00-05:00,2025-09-09 08:35:00.057085-05:00,...,124784.0,POINT (-74.04211649999999 4.6775201),2.0,POINT (-74.028923 4.704466),149,COMPLETED,POINT (-74.04211649999999 4.6775201),POINT (-74.028923 4.704466),2025-09-09,16
176,205683,297596.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 08:30:00-05:00,NaN,José Ricardo Pérez Alarcón,2025-09-09 08:00:00-05:00,2025-09-09 08:41:21.100119-05:00,...,49287.0,POINT (-74.06549079999999 4.693721),26356.0,POINT (-74.065145 4.7617788),149,COMPLETED,POINT (-74.06549079999999 4.693721),POINT (-74.065145 4.7617788),2025-09-09,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,207239,299313.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:00:00-05:00,NaN,Jhon Fredy Grajales,2025-09-09 15:30:00-05:00,2025-09-09 16:04:03.477625-05:00,...,125213.0,POINT (-74.0766101 4.6601203),125214.0,POINT (-74.0528801 4.6658237),149,COMPLETED,POINT (-74.0766101 4.6601203),POINT (-74.0528801 4.6658237),2025-09-09,16
252,207240,299314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 18:00:00-05:00,NaN,Julio Cesar Parra,2025-09-09 17:30:00-05:00,2025-09-09 18:41:11.377355-05:00,...,10.0,POINT (-74.075015 4.69416),109253.0,POINT (-74.0101379 4.932499299999999),149,COMPLETED,POINT (-74.075015 4.69416),POINT (-74.0101379 4.932499299999999),2025-09-09,16
253,207250,299326.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:30:00-05:00,NaN,Ancisar Echavarría,2025-09-09 16:00:00-05:00,2025-09-09 16:34:16.953142-05:00,...,20597.0,POINT (-74.05576889999999 4.712595600000001),125222.0,POINT (-74.0311353 4.705077),149,COMPLETED,POINT (-74.05576889999999 4.712595600000001),POINT (-74.0311353 4.705077),2025-09-09,16
254,207258,299334.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:30:00-05:00,NaN,Manuel Romero,2025-09-09 16:00:00-05:00,2025-09-09 16:40:42.585106-05:00,...,1960.0,POINT (-74.03740167617798 4.809636179915433),125225.0,POINT (-74.1018383 4.8098502),149,COMPLETED,POINT (-74.03740167617798 4.809636179915433),POINT (-74.1018383 4.8098502),2025-09-09,16


In [11]:
labors_hist_df[(labors_hist_df['city']=='149') &
              (labors_hist_df['schedule_date'].dt.day==9) &
              (labors_hist_df['labor_category']=='VEHICLE_TRANSPORTATION')]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
172,201820,293454.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 06:00:00-05:00,NaN,11712.0,2025-09-09 05:30:00-05:00,2025-09-09 06:03:24.033399-05:00,...,123975.0,POINT (-74.10727299999999 4.6525963),123976.0,POINT (-74.1145559 4.6716483),149,COMPLETED,POINT (-74.10727299999999 4.6525963),POINT (-74.1145559 4.6716483),2025-09-09,16
173,204964,296831.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 07:00:00-05:00,NaN,10451.0,2025-09-09 06:30:00-05:00,2025-09-09 07:03:43.853904-05:00,...,123805.0,POINT (-74.0574336 4.708063999999999),10.0,POINT (-74.075015 4.69416),149,COMPLETED,POINT (-74.0574336 4.708063999999999),POINT (-74.075015 4.69416),2025-09-09,16
174,205425,297314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 13:30:00-05:00,NaN,719.0,2025-09-09 13:00:00-05:00,2025-09-09 13:39:20.784593-05:00,...,125179.0,POINT (-74.03973859999999 4.7061332),28798.0,POINT (-74.08720430000001 4.6760977),149,FAILED,POINT (-74.03973859999999 4.7061332),POINT (-74.08720430000001 4.6760977),2025-09-09,16
175,205461,297351.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 08:30:00-05:00,NaN,6228.0,2025-09-09 08:00:00-05:00,2025-09-09 08:35:00.057085-05:00,...,124784.0,POINT (-74.04211649999999 4.6775201),2.0,POINT (-74.028923 4.704466),149,COMPLETED,POINT (-74.04211649999999 4.6775201),POINT (-74.028923 4.704466),2025-09-09,16
176,205683,297596.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 08:30:00-05:00,NaN,10500.0,2025-09-09 08:00:00-05:00,2025-09-09 08:41:21.100119-05:00,...,49287.0,POINT (-74.06549079999999 4.693721),26356.0,POINT (-74.065145 4.7617788),149,COMPLETED,POINT (-74.06549079999999 4.693721),POINT (-74.065145 4.7617788),2025-09-09,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,207239,299313.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:00:00-05:00,NaN,6229.0,2025-09-09 15:30:00-05:00,2025-09-09 16:04:03.477625-05:00,...,125213.0,POINT (-74.0766101 4.6601203),125214.0,POINT (-74.0528801 4.6658237),149,COMPLETED,POINT (-74.0766101 4.6601203),POINT (-74.0528801 4.6658237),2025-09-09,16
252,207240,299314.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 18:00:00-05:00,NaN,68168.0,2025-09-09 17:30:00-05:00,2025-09-09 18:41:11.377355-05:00,...,10.0,POINT (-74.075015 4.69416),109253.0,POINT (-74.0101379 4.932499299999999),149,COMPLETED,POINT (-74.075015 4.69416),POINT (-74.0101379 4.932499299999999),2025-09-09,16
253,207250,299326.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:30:00-05:00,NaN,2331.0,2025-09-09 16:00:00-05:00,2025-09-09 16:34:16.953142-05:00,...,20597.0,POINT (-74.05576889999999 4.712595600000001),125222.0,POINT (-74.0311353 4.705077),149,COMPLETED,POINT (-74.05576889999999 4.712595600000001),POINT (-74.0311353 4.705077),2025-09-09,16
254,207258,299334.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-09 16:30:00-05:00,NaN,14671.0,2025-09-09 16:00:00-05:00,2025-09-09 16:40:42.585106-05:00,...,1960.0,POINT (-74.03740167617798 4.809636179915433),125225.0,POINT (-74.1018383 4.8098502),149,COMPLETED,POINT (-74.03740167617798 4.809636179915433),POINT (-74.1018383 4.8098502),2025-09-09,16


## TODO

In [16]:
labors_real_df[(labors_real_df['city']=='149') &
              (labors_real_df['schedule_date'].dt.day==9) &
              (labors_real_df['alfred']==35582.0)].sort_values(by=['labor_start_date',
                                                                   'labor_end_date'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,address_id,address_point,address_name,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service
803,207165,299217.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 11:00:00-05:00,NaN,35582.0,2025-09-09 11:54:00-05:00,2025-09-09 11:54:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,94865.0,POINT (-74.02999919999999 4.8533407),67455.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
787,206904,298934.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 10:00:00-05:00,NaN,35582.0,2025-09-09 11:54:00-05:00,2025-09-09 12:56:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
791,206926,298956.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 12:00:00-05:00,NaN,35582.0,2025-09-09 12:56:00-05:00,2025-09-09 17:42:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED
820,207264,299340.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 15:00:00-05:00,NaN,35582.0,2025-09-09 15:00:00-05:00,2025-09-09 15:00:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,47155.0,POINT (-74.0625098 4.686226599999999),27072.0,POINT (-73.95106249999999 4.7905625),149,COMPLETED
790,206921,298951.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 14:00:00-05:00,NaN,35582.0,2025-09-09 17:42:00-05:00,2025-09-09 17:43:00-05:00,55386.0,POINT (-74.0844326 4.7205401),Casa,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED


In [19]:
labors_hist_df[(labors_hist_df['city']=='149') &
              (labors_hist_df['schedule_date'].dt.day==9) &
              (labors_hist_df['alfred']==35582.0)].sort_values(by=['historic_start',
                                                                   'historic_end'])

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
222,206904,298934.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 10:00:00-05:00,NaN,35582.0,2025-09-09 09:30:00-05:00,2025-09-09 10:27:00.862781-05:00,...,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED,POINT (-74.0299644 4.8484015),POINT (-74.07620949999999 4.6931322),2025-09-09,16
238,207165,299217.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 11:00:00-05:00,NaN,35582.0,2025-09-09 11:04:05.145171-05:00,2025-09-09 12:01:53.356964-05:00,...,94865.0,POINT (-74.02999919999999 4.8533407),67455.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED,POINT (-74.02999919999999 4.8533407),POINT (-74.07620949999999 4.6931322),2025-09-09,16
225,206921,298951.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 14:00:00-05:00,NaN,35582.0,2025-09-09 13:30:00-05:00,2025-09-09 14:27:00.862781-05:00,...,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED,POINT (-74.0299644 4.8484015),POINT (-74.07620949999999 4.6931322),2025-09-09,16
255,207264,299340.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 15:00:00-05:00,NaN,35582.0,2025-09-09 14:30:25.027361-05:00,2025-09-09 15:25:50.044391-05:00,...,47155.0,POINT (-74.0625098 4.686226599999999),27072.0,POINT (-73.95106249999999 4.7905625),149,COMPLETED,POINT (-74.0625098 4.686226599999999),POINT (-73.95106249999999 4.7905625),2025-09-09,16
226,206926,298956.0,11.0,Trailer transport,VEHICLE_TRANSPORTATION,2025-09-09 12:00:00-05:00,NaN,<NA>,NaT,NaT,...,113943.0,POINT (-74.0299644 4.8484015),113944.0,POINT (-74.07620949999999 4.6931322),149,COMPLETED,POINT (-74.0299644 4.8484015),POINT (-74.07620949999999 4.6931322),2025-09-09,16


In [5]:
moves_algo_df[(moves_algo_df['city']=='149') &
              (moves_algo_df['schedule_date'].dt.day==8) &
              (moves_algo_df['labor_category']=='DRIVER_MOVE')].groupby(['service_id']).agg({'labor_id':'count'}).sort_values(['labor_id'], ascending=False)

,labor_id
service_id,
256619,2
256887,2
256917,2
257424,2
257512,2
256082,1
256393,1
256525,1
255935,1


In [6]:
moves_hist_df[(moves_hist_df['city']=='149') &
              (moves_hist_df['schedule_date'].dt.day==8) &
              (moves_hist_df['labor_category']=='DRIVER_MOVE')].groupby(['service_id']).agg({'labor_id':'count'}).sort_values(['labor_id'],ascending=False)

,labor_id
service_id,
256917,2
256619,2
256887,2
257424,2
257512,2
253024,1
256645,1
256535,1
256381,1


In [9]:
labors_real_df[labors_real_df['service_id']==256619]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,alfred,labor_start_date,labor_end_date,address_id,address_point,address_name,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service
926,256619,352879.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,211.0,2025-09-08 15:01:41-05:00,2025-09-08 15:42:11-05:00,56465.0,POINT (-74.0877144 4.7271318),Casa,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED
927,256619,352880.0,3.0,Wash and Polish,WASH_AND_POLISH,2025-09-08 15:00:00-05:00,897.0,NaN,2025-09-08 15:42:11-05:00,2025-09-08 16:18:30-05:00,67252.0,POINT (-74.07305460423231 4.713989646958405),Agility Car Wash,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED
928,256619,352881.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,211.0,2025-09-08 16:18:30-05:00,2025-09-08 16:55:14-05:00,56465.0,POINT (-74.0877144 4.7271318),Casa,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED


In [10]:
labors_hist_df[labors_hist_df['service_id']==256619]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,historic_driver,historic_start,historic_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
22,256619,352879.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,211.0,2025-09-08 14:30:00-05:00,2025-09-08 15:03:16.660397-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.0877144 4.7271318),POINT (-74.07305460423231 4.713989646958405),2025-09-08,20
23,256619,352880.0,3.0,Wash and Polish,WASH_AND_POLISH,2025-09-08 15:00:00-05:00,897.0,<NA>,2025-09-08 15:03:16.660397-05:00,2025-09-08 16:19:46.160397-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),2025-09-08,20
24,256619,352881.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,211.0,2025-09-08 16:19:46.160397-05:00,2025-09-08 16:53:02.820794-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.07305460423231 4.713989646958405),POINT (-74.0877144 4.7271318),2025-09-08,20


In [11]:
labors_algo_df[labors_algo_df['service_id']==256619]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
22,256619,352879.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,Iván Darío Pinta,2025-09-08 14:30:00-05:00,2025-09-08 15:03:16.660397-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.0877144 4.7271318),POINT (-74.07305460423231 4.713989646958405),2025-09-08,20
23,256619,352880.0,3.0,Wash and Polish,WASH_AND_POLISH,2025-09-08 15:00:00-05:00,897.0,<NA>,2025-09-08 15:03:16.660397-05:00,2025-09-08 16:19:46.160397-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),2025-09-08,20
24,256619,352881.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2025-09-08 15:00:00-05:00,NaN,Yeisson Beltran Garzon,2025-09-08 16:19:46.160397-05:00,2025-09-08 16:53:02.820794-05:00,...,132564.0,POINT (-74.0691437 4.741131899999999),132564.0,POINT (-74.0691437 4.741131899999999),149,COMPLETED,POINT (-74.07305460423231 4.713989646958405),POINT (-74.0877144 4.7271318),2025-09-08,20


In [12]:
moves_algo_df[moves_algo_df['service_id']==256619].sort_values(['actual_start']).sort_values(['actual_start'])

,service_id,labor_id,labor_name,labor_category,assigned_driver,schedule_date,actual_start,actual_end,start_point,end_point,distance_km,duration_min,city,date
211,256619,352881_free,FREE_TIME,FREE_TIME,Yeisson Beltran Garzon,2025-09-08 15:00:00-05:00,2025-09-08 11:34:55.266425-05:00,2025-09-08 16:13:30.688703-05:00,POINT (-74.0468983 4.7033939),POINT (-74.0468983 4.7033939),0.000000,278.6,149,2025-09-08
162,256619,352879_free,FREE_TIME,FREE_TIME,Iván Darío Pinta,2025-09-08 15:00:00-05:00,2025-09-08 11:35:17.905676-05:00,2025-09-08 14:23:01.794949-05:00,POINT (-74.0780903 4.6972938),POINT (-74.0780903 4.6972938),0.000000,167.7,149,2025-09-08
163,256619,352879_move,DRIVER_MOVE,DRIVER_MOVE,Iván Darío Pinta,2025-09-08 15:00:00-05:00,2025-09-08 14:23:01.794949-05:00,2025-09-08 14:30:00-05:00,POINT (-74.0780903 4.6972938),POINT (-74.0877144 4.7271318),3.485042,7.0,149,2025-09-08
164,256619,352879,Alfred Initial Transport,VEHICLE_TRANSPORTATION,Iván Darío Pinta,2025-09-08 15:00:00-05:00,2025-09-08 14:30:00-05:00,2025-09-08 15:03:16.660397-05:00,POINT (-74.0877144 4.7271318),POINT (-74.07305460423231 4.713989646958405),NaN,33.3,149,2025-09-08
186,256619,352880,Wash and Polish,WASH_AND_POLISH,<NA>,2025-09-08 15:00:00-05:00,2025-09-08 15:03:16.660397-05:00,2025-09-08 16:19:46.160397-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),NaN,76.5,149,2025-09-08
212,256619,352881_move,DRIVER_MOVE,DRIVER_MOVE,Yeisson Beltran Garzon,2025-09-08 15:00:00-05:00,2025-09-08 16:13:30.688703-05:00,2025-09-08 16:19:46.160397-05:00,POINT (-74.0468983 4.7033939),POINT (-74.07305460423231 4.713989646958405),3.128931,6.3,149,2025-09-08
213,256619,352881,Alfred Transport,VEHICLE_TRANSPORTATION,Yeisson Beltran Garzon,2025-09-08 15:00:00-05:00,2025-09-08 16:19:46.160397-05:00,2025-09-08 16:53:02.820794-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.0877144 4.7271318),NaN,33.3,149,2025-09-08


In [13]:
moves_hist_df[moves_hist_df['service_id']==256619].sort_values(['historic_start'])

,service_id,labor_id,labor_name,labor_category,historic_driver,schedule_date,historic_start,historic_end,start_point,end_point,distance_km,duration_min,city,date
128,256619,352879_free,FREE_TIME,FREE_TIME,211.0,2025-09-08 15:00:00-05:00,2025-09-08 11:17:25.318440-05:00,2025-09-08 14:21:11.114845-05:00,POINT (-74.065413 4.759951),POINT (-74.065413 4.759951),0.000000,183.8,149,2025-09-08
129,256619,352879_move,DRIVER_MOVE,DRIVER_MOVE,211.0,2025-09-08 15:00:00-05:00,2025-09-08 14:21:11.114845-05:00,2025-09-08 14:30:00-05:00,POINT (-74.065413 4.759951),POINT (-74.0877144 4.7271318),4.407376,8.8,149,2025-09-08
130,256619,352879,Alfred Initial Transport,VEHICLE_TRANSPORTATION,211.0,2025-09-08 15:00:00-05:00,2025-09-08 14:30:00-05:00,2025-09-08 15:03:16.660397-05:00,POINT (-74.0877144 4.7271318),POINT (-74.07305460423231 4.713989646958405),NaN,33.3,149,2025-09-08
146,256619,352880,Wash and Polish,WASH_AND_POLISH,<NA>,2025-09-08 15:00:00-05:00,2025-09-08 15:03:16.660397-05:00,2025-09-08 16:19:46.160397-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),NaN,76.5,149,2025-09-08
168,256619,352881_free,FREE_TIME,FREE_TIME,211.0,2025-09-08 15:00:00-05:00,2025-09-08 15:03:16.660397-05:00,2025-09-08 16:19:46.160397-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),0.000000,76.5,149,2025-09-08
170,256619,352881,Alfred Transport,VEHICLE_TRANSPORTATION,211.0,2025-09-08 15:00:00-05:00,2025-09-08 16:19:46.160397-05:00,2025-09-08 16:53:02.820794-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.0877144 4.7271318),NaN,33.3,149,2025-09-08
169,256619,352881_move,DRIVER_MOVE,DRIVER_MOVE,211.0,2025-09-08 15:00:00-05:00,2025-09-08 16:19:46.160397-05:00,2025-09-08 16:19:46.160397-05:00,POINT (-74.07305460423231 4.713989646958405),POINT (-74.07305460423231 4.713989646958405),0.000000,0.0,149,2025-09-08


In [14]:
metrics_real_df = collect_vt_metrics_range( labors_hist_df, 
                                            start_date='2025-09-08', 
                                            end_date='2025-09-13',
                                            dist_dict=dist_dict,
                                            workday_hours=9,
                                            city_code='149',
                                            skip_weekends=False,
                                            assignment_type='historic'
                                            )

metrics_alg_df = collect_vt_metrics_range(  labors_algo_df, 
                                            start_date='2025-09-08', 
                                            end_date='2025-09-13',
                                            dist_dict=dist_dict,                                                                    
                                            workday_hours=9,
                                            city_code='149',
                                            skip_weekends=False,
                                            assignment_type='algorithm'              
                                            )

# df_dist_art = load_artificial_instance(data_path, labors_raw_df)
day_str = '2025-09-08'
summary_real, (df_plot_hist, driver_col) = show_day_report_dayonly(labors_hist_df, day_str=day_str, city_code ='149', 
                                                         dist_dict=dist_dict, return_plotting_df=True,
                                                         assignment_type='historic')
summary_algo, (df_plot_algo, driver_col) = show_day_report_dayonly(labors_algo_df, day_str=day_str, city_code ='149', 
                                                         dist_dict=dist_dict, return_plotting_df=True,
                                                         assignment_type='algorithm')

Reporte del día: 2025-09-08


,Métrica,Valor
0,Labores VEHICLE_TRANSPORTATION (día filtrado),75
1,Conductores únicos (alfred),20
2,Labores por conductor,3.75
3,Utilización promedio conductores,30.8%
4,Distancia total recorrida (km),1347.92
5,Tiempo extra total (min),897.5
6,Tiempo extra total (horas),14.96
7,Tiempo extra temprano (min),791.7
8,Tiempo extra tarde (min),105.8


Reporte del día: 2025-09-08


,Métrica,Valor
0,Labores VEHICLE_TRANSPORTATION (día filtrado),75
1,Conductores únicos (alfred),20
2,Labores por conductor,3.75
3,Utilización promedio conductores,30.8%
4,Distancia total recorrida (km),1347.92
5,Tiempo extra total (min),882.4
6,Tiempo extra total (horas),14.71
7,Tiempo extra temprano (min),791.7
8,Tiempo extra tarde (min),90.6


In [15]:
plot_gantt_labors_by_driver(df_plot_hist, day_str=day_str, driver_col='historic_driver')
plot_gantt_labors_by_driver(df_plot_algo, day_str=day_str, driver_col='assigned_driver')

In [16]:
plot_results(labors_hist_df[(labors_hist_df['schedule_date'].dt.day==8) & 
                (labors_hist_df['city']=='149')], 
             moves_hist_df[(moves_hist_df['schedule_date'].dt.day==8) & 
               (moves_hist_df['city']=='149')], 
             day_str, TIEMPO_GRACIA, assignment_type='historic')

/Users/juanbeta/Library/CloudStorage/GoogleDrive-juan.beta98@gmail.com/My Drive/Work/Alfred/Alfred/code/src/plotting.py:265: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/juanbeta/Library/CloudStorage/GoogleDrive-juan.beta98@gmail.com/My Drive/Work/Alfred/Alfred/code/src/plotting.py:266: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [17]:
plot_results(labors_algo_df[(labors_algo_df['schedule_date'].dt.day==8) & 
                (labors_algo_df['city']=='149')], 
             moves_algo_df[(moves_algo_df['schedule_date'].dt.day==8) & 
               (moves_algo_df['city']=='149')], 
             day_str, TIEMPO_GRACIA, assignment_type='algorithm')

In [10]:
labors_algo_df[labors_algo_df['service_id']==256535]

,service_id,labor_id,labor_type,labor_name,labor_category,schedule_date,shop,assigned_driver,actual_start,actual_end,...,start_address_id,start_address_point,end_address_id,end_address_point,city,state_service,map_start_point,map_end_point,date,n_drivers
18,256535,352784.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,2025-09-08 08:00:00-05:00,NaN,Jhon Fredy Grajales,2025-09-08 07:30:00-05:00,2025-09-08 14:00:07.316986-05:00,...,140878.0,POINT (-75.5751567 6.238464),42949.0,POINT (-74.1114247 4.6451577),149,COMPLETED,POINT (-75.5751567 6.238464),POINT (-74.1114247 4.6451577),2025-09-08,20
19,256535,353842.0,16.0,Tanqueo Terpel (Corriente),OTHER_LABORS,2025-09-08 08:00:00-05:00,706.0,<NA>,2025-09-08 14:00:07.316986-05:00,2025-09-08 14:04:18.495557-05:00,...,140878.0,POINT (-75.5751567 6.238464),42949.0,POINT (-74.1114247 4.6451577),149,COMPLETED,POINT (-75.6372042 6.0977),POINT (-75.6372042 6.0977),2025-09-08,20
20,256535,353843.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,2025-09-08 08:00:00-05:00,NaN,Manuel Romero,2025-09-08 14:04:18.495557-05:00,2025-09-08 20:34:25.812543-05:00,...,140878.0,POINT (-75.5751567 6.238464),42949.0,POINT (-74.1114247 4.6451577),149,COMPLETED,POINT (-75.5751567 6.238464),POINT (-74.1114247 4.6451577),2025-09-08,20


In [16]:
labors_real_df[(labors_real_df['schedule_date'].dt.day==8) &
               (labors_real_df['city']=='149') &
               (labors_real_df['schedule_date'].dt.hour<=7)]

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,shop,created_at,schedule_date,client_type,paying_customer,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name
903,252699,348654.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,52988.0,2025-06-09 16:19:29.440000-05:00,2025-09-08 06:39:00-05:00,2025-09-08 07:58:00-05:00,5366.0,NaN,2025-06-09 16:19:29.298000-05:00,2025-09-08 07:00:00-05:00,B2B,49196.0,FAILED,139688.0,POINT (-74.0699599 4.7038514),2.0,POINT (-74.028923 4.704466),149,93651.0,POINT (-74.08878779999999 4.7315578),ALFRED
905,253604,349658.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,35000.0,2025-06-11 10:29:29.541000-05:00,2025-09-08 09:18:00-05:00,2025-09-08 10:51:00-05:00,493.0,NaN,2025-06-11 10:29:29.466000-05:00,2025-09-08 07:30:00-05:00,B2C,71629.0,COMPLETED,135446.0,POINT (-74.05544019999999 4.6851802),135446.0,POINT (-74.05544019999999 4.6851802),149,56342.0,POINT (-74.1419617 4.7106052),Casa
906,253801,349882.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-11 16:24:40.927000-05:00,2025-09-08 07:00:00-05:00,2025-09-08 07:51:00-05:00,211.0,NaN,2025-06-11 16:24:40.851000-05:00,2025-09-08 07:00:00-05:00,B2B,66600.0,COMPLETED,140039.0,POINT (-74.06717499999999 4.700141),140040.0,POINT (-74.0664901 4.6872453),149,56465.0,POINT (-74.0877144 4.7271318),Casa
908,254732,350872.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-13 09:28:55.616000-05:00,2025-09-08 06:10:00-05:00,2025-09-08 08:05:00-05:00,4006.0,NaN,2025-06-13 09:28:55.547000-05:00,2025-09-08 07:00:00-05:00,B2B,76474.0,COMPLETED,140289.0,POINT (-74.1171214 4.7017318),140290.0,POINT (-74.11402149999999 4.6720214),149,NaN,NaN,NaN
909,255227,351399.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,49485.0,2025-06-14 09:18:19.226000-05:00,2025-09-08 07:00:00-05:00,2025-09-08 08:15:00-05:00,69861.0,NaN,2025-06-14 09:18:19.158000-05:00,2025-09-08 07:00:00-05:00,B2B,57674.0,COMPLETED,86927.0,POINT (-74.1192667 4.620530599999999),1460.0,POINT (-74.1127455 4.6451826),149,125737.0,POINT (-74.18022189999999 4.624224),Apartamento
910,255744,351951.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,55663.0,2025-06-16 10:33:39.632000-05:00,2025-09-08 06:55:00-05:00,2025-09-08 08:25:00-05:00,719.0,NaN,2025-06-16 10:33:39.534000-05:00,2025-09-08 07:00:00-05:00,B2B,66600.0,COMPLETED,140621.0,POINT (-74.0840223 4.693518399999999),140622.0,POINT (-74.0664901 4.6872453),149,9575.0,POINT (-74.1474839 4.6118768),Apartamento
911,255781,351992.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,49485.0,2025-06-16 11:24:00.721000-05:00,2025-09-08 06:20:00-05:00,2025-09-08 07:43:00-05:00,3214.0,NaN,2025-06-16 11:24:00.645000-05:00,2025-09-08 07:00:00-05:00,B2B,64404.0,COMPLETED,140642.0,POINT (-74.04058150000002 4.7038041),33851.0,POINT (-74.06467649999999 4.7610987),149,NaN,NaN,NaN
913,255839,352052.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,57789.0,2025-06-16 12:30:11.537000-05:00,2025-09-08 07:25:00-05:00,2025-09-08 08:17:00-05:00,5832.0,NaN,2025-06-16 12:30:11.471000-05:00,2025-09-08 07:30:00-05:00,B2B,71684.0,COMPLETED,140667.0,POINT (-74.0504031 4.757521),140668.0,POINT (-74.02394079999999 4.728333399999999),149,98373.0,POINT (-74.0805306 4.755374499999999),casa


In [7]:
labors_raw_df[labors_raw_df['service_id']==257512]

,service_id,labor_id,labor_type,labor_name,labor_category,labor_price,labor_created_at,labor_start_date,labor_end_date,alfred,...,paying_customer,state_service,start_address_id,start_address_point,end_address_id,end_address_point,city,address_id,address_point,address_name
100620,257512,353831.0,12.0,Alfred Initial Transport,VEHICLE_TRANSPORTATION,35000.0,2025-06-19 10:02:39.449000-05:00,2025-06-19 14:35:23-05:00,2025-06-19 16:38:25-05:00,10451.0,...,52406.0,COMPLETED,82371.0,POINT (-74.06494219999999 4.6357845),82371.0,POINT (-74.06494219999999 4.6357845),149,16688.0,POINT (-74.0202608 4.6664181),casa
100621,257512,353832.0,3.0,Wash and Polish,WASH_AND_POLISH,28500.0,2025-06-19 10:02:39.524000-05:00,2025-06-19 16:38:25-05:00,2025-06-19 16:38:29-05:00,NaN,...,52406.0,COMPLETED,82371.0,POINT (-74.06494219999999 4.6357845),82371.0,POINT (-74.06494219999999 4.6357845),149,72261.0,POINT (-74.1089748 4.6597407),Deluxe CWF
100622,257512,353833.0,2.0,Alfred Transport,VEHICLE_TRANSPORTATION,35000.0,2025-06-19 10:02:39.586000-05:00,2025-06-19 16:38:29-05:00,2025-06-19 17:45:02-05:00,10451.0,...,52406.0,COMPLETED,82371.0,POINT (-74.06494219999999 4.6357845),82371.0,POINT (-74.06494219999999 4.6357845),149,16688.0,POINT (-74.0202608 4.6664181),casa


In [10]:
labors_raw_df[labors_raw_df['alfred']==10451]['address_point'].unique()

array(['POINT (-74.0202608 4.6664181)'], dtype=object)